In [ ]:
import numpy as np
from  as sci



ModuleNotFoundError: No module named 'scipy'

## Parameter estimation for a linear operator using Gaussian processes


Assumptions about the linear operator:

$\mathcal{L}_x^\phi u(x) = f(x)$

$u(x) \sim \mathcal{GP}(0, k_{uu}(x,x',\theta))$

$f(x) \sim \mathcal{GP}(0, k_{ff}(x,x',\theta,\phi))$

$y_u = u(X_u) + \epsilon_u; \epsilon_u \sim \mathcal{N}(0, \sigma_u^2I)$

$y_f = f(X_f) + \epsilon_f; \epsilon_f \sim \mathcal{N}(0, \sigma_f^2I)$

Taking a simple operator as example:

$\mathcal{L}_x^\phi := \phi \cdot + \frac{d}{dx}\cdot$

$u(x) = sin(x)$

$f(x) = \phi sin(x) + cos(x)$

Problem at hand:

Given $\{X_u, y_u\}$ and $\{X_f, y_f\}$, estimate $\phi$.


#### step 1: simulate data


Use $\phi = 2$


In [1]:
t = 1
u0 = 5
dt = 0.1
x_u = np.linspace(0,2*np.pi,10)
y_u = (4*u0/np.pi)*np.sin(np.pi*x)*np.exp(-np.square(np.pi)*t)
x_f = np.linspace(0,2*np.pi, 10)
y_f = (4*u0/np.pi)*np.sin(np.pi*x)*np.exp(-np.square(np.pi)*t)+dt*3*(4*u0*np.pi)*np.sin(np.pi*x)*np.exp(-np.square(np.pi)*t)

NameError: name 'np' is not defined

#### step 2: create covariance matrix



This step uses information about $\mathcal{L}_x^\phi$ but not about $u(x)$ or $f(x)$.

$k_{uu}(x_i, x_j; \theta) = \theta exp(-\frac{1}{2}(x_i-x_j)^2)$


In [11]:
def kuu(x, theta):
    n = x_u.size
    a = np.repeat(x_u, n).reshape(n, n)
    return theta * np.exp(-0.5*np.square(a - a.transpose()))

$k_{ff}(x_i,x_j;\theta,\phi) \\
= \mathcal{L}_{x_i}^\phi \mathcal{L}_{x_j}^\phi k_{uu}(x_i, x_j; \theta) \\
= \mathcal{L}_{x_i}^\phi \mathcal{L}_{x_j}^\phi \left( \theta exp(-\frac{1}{2}(x_i-x_j)^2) \right] \\
= \mathcal{L}_{x_i}^\phi \left[ \theta exp(-\frac{1}{2}(x_i-x_j)^2)\left(\phi + (-\frac{1}{2})2(x_i-x_j)(-1) \right) \right] \\
= \mathcal{L}_{x_i}^\phi \left[\theta exp(-\frac{1}{2}(x_i-x_j)^2)(\phi + x_i - x_j) \right] \\
= \phi \theta exp(-\frac{1}{2}(x_i-x_j)^2)(\phi+x_i-x_j) + \theta exp(-\frac{1}{2}(x_i-x_j)^2)\left[ -\frac{1}{2}2(x_i-x_j)(\phi+x_i-x_j) + 1 \right] \\
= \theta exp(-\frac{1}{2}(x_i-x_j)^2)\left[ \phi^2 - (x_i-x_j)^2 + 1 \right]$

In [12]:
def kff(x, theta, phi):
    n = x_u.size
    a = np.repeat(x_u, n).reshape(n, n)
    return np.multiply(kuu(x, theta), (phi*phi - np.square(a - a.transpose()) + 1) )

$k_{fu}(x_i,x_j;\theta,\phi) \\
= \mathcal{L}_{x_i}^\phi k_{uu}(x_i, x_j; \theta) \\
= \mathcal{L}_{x_i}^\phi \left[ \theta exp(-\frac{1}{2}(x_i-x_j)^2) \right] \\
= \theta exp(-\frac{1}{2}(x_i-x_j)^2) \left[ (-\frac{1}{2})2(x_i-x_j) + \phi \right] \\
= \theta exp(-\frac{1}{2}(x_i-x_j)^2)(\phi-x_i+x_j)$

In [13]:
def kfu(x1, x2, theta, phi):
    n1 = x1.size
    n2 = x2.size
    a1 = np.repeat(x1, n2).reshape(n1, n2)
    a2 = np.repeat(x2, n1).reshape(n2, n1).transpose()
    return np.multiply(theta * np.exp(-0.5*np.square(a1 - a2)), (phi - a1 + a2))

$k_{uf}(x_i,x_j;\theta,\phi) \\
= \mathcal{L}_{x_j}^\phi k_{uu}(x_i, x_j; \theta) \\
= \mathcal{L}_{x_j}^\phi \left[ \theta exp(-\frac{1}{2}(x_i-x_j)^2) \right] \\
= \theta exp(-\frac{1}{2}(x_i-x_j)^2) \left[ (-\frac{1}{2})2(x_i-x_j)(-1) + \phi \right]\\
= \theta exp(-\frac{1}{2}(x_i-x_j)^2)(\phi+x_i-x_j)$

In [14]:
def kuf(x1, x2, theta, phi):
    n1 = x1.size
    n2 = x2.size
    a1 = np.repeat(x1, n2).reshape(n1, n2)
    a2 = np.repeat(x2, n1).reshape(n2, n1).transpose()
    return np.multiply(theta * np.exp(-0.5*np.square(a1 - a2)), (phi + a1 - a2))

#### step 3: define negative log marginal likelihood  



$K = \begin{bmatrix}
k_{uu}(X_u, X_u; \theta) + \sigma_u^2I & k_{uf}(X_u, X_f; \theta, \phi) \\
k_{fu}(X_f, X_u; \theta, \phi) & k_{ff}(X_f, X_f; \theta, \phi) + \sigma_f^2I
\end{bmatrix}$

For simplicity, assume $\sigma_u = \sigma_f$.

$\mathcal{NLML} = \frac{1}{2} \left[ log|K| + y^TK^{-1}y + Nlog(2\pi) \right]$

where $y = \begin{bmatrix}
y_u \\
y_f
\end{bmatrix}$

In [15]:
def nlml(params, x1, x2, y1, y2, s):
    K = np.block([
        [kuu(x1, params[0]) + s*np.identity(x1.size), kuf(x1, x2, params[0], params[1])],
        [kfu(x1, x2, params[0], params[1]), kff(x2, params[0], params[1]) + s*np.identity(x2.size)]
    ])
    y = np.concatenate((y1, y2))
    val = 0.5*(np.log(abs(np.linalg.det(K))) + np.mat(y) * np.linalg.inv(K) * np.mat(y).T)
    return val.item(0)

In [16]:
nlml((1, 2), x_u, x_f, y_u, y_f, 1e-6)

287207973709.46

#### step 4: optimise hyperparameters


In [17]:
minimize(nlml, np.random.rand(2), args=(x_u, x_f, y_u, y_f, 1e-6), method="Nelder-Mead")

 final_simplex: (array([[184.21790874,  45.00054242],
       [184.21783484,  45.00054242],
       [184.21797566,  45.00054242]]), array([9.65413379, 9.65413416, 9.65413423]))
           fun: 9.654133789790773
       message: 'Optimization terminated successfully.'
          nfev: 224
           nit: 110
        status: 0
       success: True
             x: array([184.21790874,  45.00054242])